In [39]:
import ssl
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

ssl._create_default_https_context = ssl._create_unverified_context
(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

In [40]:
# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
class_names = ['티셔츠', '바지', '스웨터', '드레스', '코트', '샌달', '셔츠', '운동화', '가방', '앵클부츠']

sigmoid: 결과를 0~1로 압축해줌
- binary 예측문제에 사용한다
- 마지막 노드 갯수는 1개로한다

softmax: 결과를 0~1로 압축해준다
- 카테고리가 여러개일때 예측문제에 사용한다
- 예측한 카테고리 확률을 모두 더하면 1이 나온다

relu: 모든 음수를 0으로 만들어준다
- convolution layer에서 자주 쓴다
- 이미지는 0~255(RGB) 여서 음수 나올일이 없기때문에 이미지 에서 씀

In [41]:
trainX = trainX / 255.0
testX = testX / 255.0

In [42]:
trainX = trainX.reshape( (trainX.shape[0], 28, 28, 1) )
testX = testX.reshape( (testX.shape[0], 28, 28, 1 ) )
# 흑백일경우 (x, y, 1)
# 컬러일경우 RGB들어가서 (x, y, 3)

In [43]:
# Conv2D( 이미지 갯수, 커널 사이즈 )
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D( 32, (3,3), padding="same", activation="relu", input_shape=(28, 28, 1) ), 
  tf.keras.layers.MaxPooling2D( (2,2) ),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 6272)              0         
                                                                 
 dense_36 (Dense)            (None, 64)                401472    
                                                                 
 dense_37 (Dense)            (None, 10)                650       
                                                                 
Total params: 402442 (1.54 MB)
Trainable params: 402442 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

Flatten layer
* 2차원 데이터를 1차원 데이터로 압축해줌

이미지를 flatten 했을때의 문제점
* 응용력이 없어짐, 기존 가중치의 의미가 사라짐

해결방법
* convolution layer: 이미지를 여러 버전으로 만들어줌 (여러 커널으로 각각 다른 특징을 살려서)

단순 Convolution Layer의 문제점
- 이미지가 이동했을때 응용력이 떨어짐

해결방법
- Pooling layer(Downsampling) 이미지를 추려서 축소작업

카테고리 예측문제에서 쓰이는 손실함수
* 원핫 인코딩 되어있는경우 : categorical_crossentropy
* 데이터가 정수인경우 : sparse_categorical_crossentropy

In [44]:
model.compile( loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'] )
model.fit(trainX, trainY, epochs=5)

Epoch 1/5
   8/1875 [..............................] - ETA: 13s - loss: 2.1079 - accuracy: 0.2656 

2023-05-31 15:22:42.574920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 12s 6ms/step - loss: 0.4039 - accuracy: 0.8583
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2772 - accuracy: 0.9000
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2366 - accuracy: 0.9132
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2080 - accuracy: 0.9237
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1838 - accuracy: 0.9322


In [45]:
score = model.evaluate(testX, testY)
print(score)

 11/313 [>.............................] - ETA: 1s - loss: 0.2559 - accuracy: 0.9261 

2023-05-31 15:23:41.353890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 4ms/step - loss: 0.2682 - accuracy: 0.9047
[0.2682349979877472, 0.904699981212616]
